In [3]:
# Packages
import os
import cv2
import numpy as np
from skimage import feature
import random
import matplotlib.pyplot as plt

from scipy import signal
from scipy.fft import fft, fftfreq
import pywt 
from skimage import measure
import matplotlib.pyplot as plt

Assigning numerical labels

In [4]:
class SpiralAnalyzer:
    def __init__(self, image_size=1024):
        self.image_size = image_size
        
    def preprocess_image(self, image):
        """Preprocess the input image."""
        # Convert to grayscale if needed
        if len(image.shape) == 3:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Ensure correct size
        if image.shape != (self.image_size, self.image_size):
            image = cv2.resize(image, (self.image_size, self.image_size))
        
        # Threshold to binary
        _, binary = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY_INV)
        return binary

    def find_start_point(self, binary_image):
        """Find the most likely starting point of the spiral."""
        # Assume the spiral starts from the outer part
        contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if not contours:
            return None
        
        # Find the largest contour
        largest_contour = max(contours, key=cv2.contourArea)
        
        # Find the point farthest from the center
        center = np.array([binary_image.shape[1]/2, binary_image.shape[0]/2])
        max_dist = 0
        start_point = None
        
        for point in largest_contour[:, 0, :]:
            dist = np.linalg.norm(point - center)
            if dist > max_dist:
                max_dist = dist
                start_point = point
                
        return start_point

    def extract_spiral_coordinates(self, binary_image):
        """Extract coordinates of the spiral."""
        # Find non-zero points (spiral pixels)
        y_coords, x_coords = np.nonzero(binary_image)
        
        # Convert to polar coordinates
        center_x = binary_image.shape[1] / 2
        center_y = binary_image.shape[0] / 2
        
        x = x_coords - center_x
        y = y_coords - center_y
        
        r = np.sqrt(x**2 + y**2)
        theta = np.arctan2(y, x)
        
        # Sort by theta to unwrap the spiral
        sort_idx = np.argsort(theta)
        r = r[sort_idx]
        theta = theta[sort_idx]
        
        return r, theta

    def compute_frequency_features(self, r, theta, sampling_rate=100):
        """Compute frequency domain features."""
        # Normalize radius to remove trend
        r_normalized = r - np.mean(r)
        
        # Compute FFT
        n = len(r_normalized)
        fft_vals = fft(r_normalized)
        freqs = fftfreq(n, 1/sampling_rate)
        
        # Get positive frequencies only
        pos_mask = freqs >= 0
        freqs = freqs[pos_mask]
        fft_vals = np.abs(fft_vals[pos_mask])
        
        # Calculate PSD
        psd = np.abs(fft_vals)**2 / n
        
        # Extract specific frequency bands
        essential_tremor_mask = (freqs >= 4) & (freqs <= 8)
        parkinsonian_mask = (freqs >= 3) & (freqs <= 7)
        
        features = {
            'peak_frequency': freqs[np.argmax(psd)],
            'peak_amplitude': np.max(psd),
            'essential_tremor_power': np.sum(psd[essential_tremor_mask]),
            'parkinsonian_power': np.sum(psd[parkinsonian_mask]),
            'frequency_spread': np.std(freqs[psd > np.max(psd)*0.1])
        }
        
        return features

    def compute_wavelet_features(self, r, scales=np.arange(1, 128)):
        """Compute wavelet transform features."""
        # Perform CWT
        wavelet = 'morl'
        coefficients, frequencies = pywt.cwt(r, scales, wavelet)
        
        # Extract features from wavelet coefficients
        features = {
            'wavelet_energy': np.mean(np.abs(coefficients)**2),
            'wavelet_entropy': -np.sum(np.abs(coefficients)**2 * np.log(np.abs(coefficients)**2 + 1e-10)),
            'max_wavelet_coef': np.max(np.abs(coefficients)),
            'std_wavelet_coef': np.std(np.abs(coefficients))
        }
        
        return features

    def compute_geometric_features(self, r, theta):
        """Compute geometric features of the spiral."""
        # Calculate derivatives for curvature
        dr = np.gradient(r)
        dtheta = np.gradient(theta)
        d2r = np.gradient(dr)
        d2theta = np.gradient(dtheta)
        
        # Compute curvature
        curvature = np.abs(r * d2theta - dr * dtheta) / (r**2 + dr**2)**1.5
        
        # Compute symmetry
        r_centered = r - np.mean(r)
        above_zero = np.sum(r_centered > 0)
        below_zero = np.sum(r_centered < 0)
        symmetry = min(above_zero, below_zero) / max(above_zero, below_zero)
        
        features = {
            'mean_curvature': np.mean(curvature),
            'std_curvature': np.std(curvature),
            'symmetry': symmetry,
            'radius_variance': np.var(r)
        }
        
        return features

    def analyze_spiral(self, image):
        """Main function to analyze spiral drawing and extract all features."""
        # Preprocess image
        binary = self.preprocess_image(image)
        
        # Find start point
        start_point = self.find_start_point(binary)
        if start_point is None:
            raise ValueError("Could not find spiral start point")
        
        # Extract coordinates
        r, theta = self.extract_spiral_coordinates(binary)
        
        # Compute all features
        frequency_features = self.compute_frequency_features(r, theta)
        wavelet_features = self.compute_wavelet_features(r)
        geometric_features = self.compute_geometric_features(r, theta)
        
        # Combine all features
        all_features = {
            **frequency_features,
            **wavelet_features,
            **geometric_features
        }
        
        return all_features

def process_image_batch(image_paths):
    """Process a batch of images and return their features."""
    analyzer = SpiralAnalyzer()
    all_results = []
    
    for path in image_paths:
        try:
            image = cv2.imread(path)
            features = analyzer.analyze_spiral(image)
            features['image_path'] = path
            all_results.append(features)
        except Exception as e:
            print(f"Error processing {path}: {str(e)}")
            
    return all_results

In [13]:
class SpiralAnalyzer:
    def __init__(self, image_size=1024):
        self.image_size = image_size
        
    def load_image(self, image_path):
        """Load and validate image file."""
        if not os.path.exists(image_path):
            raise FileNotFoundError(f"Image file not found: {image_path}")
            
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Failed to load image: {image_path}")
            
        return image
        
    def preprocess_image(self, image):
        """Preprocess the input image."""
        if image is None:
            raise ValueError("Input image is None")
            
        # Convert to grayscale if needed
        if len(image.shape) == 3:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Ensure correct size
        if image.shape != (self.image_size, self.image_size):
            image = cv2.resize(image, (self.image_size, self.image_size))
        
        # Threshold to binary
        _, binary = cv2.threshold(image, 127, 255, cv2.THRESH_BINARY_INV)
        
        # Verify that we have some non-zero pixels
        if np.sum(binary) == 0:
            raise ValueError("No spiral detected in image after thresholding")
            
        return binary

    def find_start_point(self, binary_image):
        """Find the most likely starting point of the spiral."""
        if binary_image is None:
            raise ValueError("Binary image is None")
            
        # Assume the spiral starts from the outer part
        contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if not contours:
            raise ValueError("No contours found in the image")
        
        # Find the largest contour
        largest_contour = max(contours, key=cv2.contourArea)
        
        # Find the point farthest from the center
        center = np.array([binary_image.shape[1]/2, binary_image.shape[0]/2])
        max_dist = 0
        start_point = None
        
        for point in largest_contour[:, 0, :]:
            dist = np.linalg.norm(point - center)
            if dist > max_dist:
                max_dist = dist
                start_point = point
                
        if start_point is None:
            raise ValueError("Could not determine spiral start point")
            
        return start_point

    def extract_spiral_coordinates(self, binary_image):
        """Extract coordinates of the spiral."""
        if binary_image is None:
            raise ValueError("Binary image is None")
            
        # Find non-zero points (spiral pixels)
        y_coords, x_coords = np.nonzero(binary_image)
        
        if len(x_coords) == 0 or len(y_coords) == 0:
            raise ValueError("No spiral points found in the image")
        
        # Convert to polar coordinates
        center_x = binary_image.shape[1] / 2
        center_y = binary_image.shape[0] / 2
        
        x = x_coords - center_x
        y = y_coords - center_y
        
        r = np.sqrt(x**2 + y**2)
        theta = np.arctan2(y, x)
        
        # Sort by theta to unwrap the spiral
        sort_idx = np.argsort(theta)
        r = r[sort_idx]
        theta = theta[sort_idx]
        
        return r, theta

    # [Previous feature computation methods remain the same...]
    def compute_frequency_features(self, r, theta, sampling_rate=100):
        """Compute frequency domain features."""
        if len(r) < 2:
            raise ValueError("Not enough points to compute frequency features")
            
        # Normalize radius to remove trend
        r_normalized = r - np.mean(r)
        
        # Compute FFT
        n = len(r_normalized)
        fft_vals = fft(r_normalized)
        freqs = fftfreq(n, 1/sampling_rate)
        
        # Get positive frequencies only
        pos_mask = freqs >= 0
        freqs = freqs[pos_mask]
        fft_vals = np.abs(fft_vals[pos_mask])
        
        # Calculate PSD
        psd = np.abs(fft_vals)**2 / n
        
        # Extract specific frequency bands
        essential_tremor_mask = (freqs >= 4) & (freqs <= 8)
        parkinsonian_mask = (freqs >= 3) & (freqs <= 7)
        
        features = {
            'peak_frequency': freqs[np.argmax(psd)],
            'peak_amplitude': np.max(psd),
            'essential_tremor_power': np.sum(psd[essential_tremor_mask]),
            'parkinsonian_power': np.sum(psd[parkinsonian_mask]),
            'frequency_spread': np.std(freqs[psd > np.max(psd)*0.1])
        }
        
        return features

    def analyze_spiral(self, image_path):
        """Main function to analyze spiral drawing and extract all features."""
        try:
            # Load image
            image = self.load_image(image_path)
            
            # Preprocess image
            binary = self.preprocess_image(image)
            
            # Verify binary image
            if binary is None:
                raise ValueError("Preprocessing failed to produce valid binary image")
            
            # Find start point
            start_point = self.find_start_point(binary)
            
            # Extract coordinates
            r, theta = self.extract_spiral_coordinates(binary)
            
            # Compute features
            features = {}
            
            try:
                frequency_features = self.compute_frequency_features(r, theta)
                features.update(frequency_features)
            except Exception as e:
                print(f"Warning: Failed to compute frequency features: {str(e)}")
            
            try:
                wavelet_features = self.compute_wavelet_features(r)
                features.update(wavelet_features)
            except Exception as e:
                print(f"Warning: Failed to compute wavelet features: {str(e)}")
            
            try:
                geometric_features = self.compute_geometric_features(r, theta)
                features.update(geometric_features)
            except Exception as e:
                print(f"Warning: Failed to compute geometric features: {str(e)}")
            
            if not features:
                raise ValueError("No features could be computed")
                
            return features
            
        except Exception as e:
            raise Exception(f"Analysis failed: {str(e)}")

def test_spiral_analyzer(image_path):
    """Test function to verify each step of the analysis."""
    analyzer = SpiralAnalyzer()
    
    print(f"Testing image: {image_path}")
    
    try:
        # Test image loading
        print("1. Testing image loading...")
        image = analyzer.load_image(image_path)
        print("   Image loaded successfully")
        
        # Test preprocessing
        print("2. Testing preprocessing...")
        binary = analyzer.preprocess_image(image)
        print("   Preprocessing successful")
        
        # Test start point detection
        print("3. Testing start point detection...")
        start_point = analyzer.find_start_point(binary)
        print(f"   Start point found at: {start_point}")
        
        # Test coordinate extraction
        print("4. Testing coordinate extraction...")
        r, theta = analyzer.extract_spiral_coordinates(binary)
        print(f"   Extracted {len(r)} points from spiral")
        
        # Test full analysis
        print("5. Testing full feature extraction...")
        features = analyzer.analyze_spiral(image_path)
        print("   Features extracted successfully:")
        for key, value in features.items():
            print(f"   {key}: {value}")
            
        return True
        
    except Exception as e:
        print(f"Test failed: {str(e)}")
        return False
    
if __name__ == "__main__":
    image_path = 'basicSpiral.png'  # Replace with your image path
    test_spiral_analyzer(image_path)


Testing image: basicSpiral.png
1. Testing image loading...
Test failed: Failed to load image: basicSpiral.png


In [11]:
if __name__ == "__main__":
    # Single image analysis
    try:
        analyzer = SpiralAnalyzer()
        image = cv2.imread('basicSpiral.png')  # Replace with your image path
        features = analyzer.analyze_spiral(image)
        print("Extracted features:", features)
    except Exception as e:
        print(f"Error during analysis: {str(e)}")

Error during analysis: Analysis failed: stat: path should be string, bytes, os.PathLike or integer, not NoneType
